In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow.keras import layers, models
import os

2025-11-24 11:17:49.514723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Definição das constantes - caminho do dataset e nomes das classes.

In [9]:
#ARQUIVOS
ATTRIBUTES_FILE = './dataset/attributes.csv'
LABELS_FILE = './dataset/label.csv'

CLASS_NAMES = {
    0: 'Normal',
    1: 'Charge',
    2: 'Discharge',
    3: 'Friction',
    4: 'Charge Discharge',
    5: 'Charge Friction',
    6: 'Discharge Friction',
    7: 'Charge Discharge Friction'
}


Os números gerados aleatoriamente pelo computador na verdade são pseudo-aleatórios. Assim, setar uma random seed constante faz com que os pesos iniciais, que são gerados aleatoriamente primeiro, sejam sempre iniciados com o mesmo valor aleatório. Para isso, travamos a aleatoriedade do numPy e do TensorFlow com .seed e set_seed. Isso trava os resultados da rede neural em sempre uma mesma acurácia.

42 é só um número atoa, ele na verdade vem da resposta para a Questão Fundamental da Vida, do Universo e Tudo Mais - O Guia do Mochileiro das Galáxias.

In [10]:
# Configuração para os dados aleatorios se repetirem
np.random.seed(42)
tf.random.set_seed(42)

### carrega_dados

**X.shape[0]**: número de linhas do array numPy com todos os dados de entrada. nesse caso, seria a quantidade de amostras disponíveis.

**X.shape[1]**: número de colunas do array numPy com todos os dados de entrada. nesse caso, seria a quantidade de colunas do csv de amostras. ou seja, quantas variações de tempo uma única amostra possui. 

**X**: 
| Amostra | feature_1 | feature_2 | ... | feature_201  |
|:---:|:---:|:---:|:---:|:---:|
| amostra 1 | valor_pressão_f1 | valor_pressão_f2 | ... | valor_pressão_f201 |
| amostra 2 | valor_pressão_f1 | valor_pressão_f2 | ... | valor_pressão_f201 |
| amostra 3 | valor_pressão_f1 | valor_pressão_f2 | ... | valor_pressão_f201 |
|  ...      |       ...        |         ...      | ... |           ...      |

**tf.keras.utils.to_categorical(y, num_classes=num_classes)**: converte rótulos de classes inteiros para formato one-hot encoded. é armazenado em uma array numPy. seu tamanho é linhas_amostras x nro_classes. 

**y_categorical**:
| Amostra | Neurônio 1 (Normal) | Neurônio 2 (Charge) | ... | Neurônio 8 (All Faults) |
|:---:|:---:|:---:|:---:|:---:|
| Linha 1 | 1 | 0 | ... | 0 |
| Linha 2 | 0 | 0 | ... | 1 |
| Linha 3 | 0 | 0 | ... | 0 |
|  ...    | 0 | 0 | ... | 0 |

In [11]:
def carrega_dados(attr_caminho, lbl_caminho):

    print(f"Lendo atributos: {attr_caminho}")
    print(f"Lendo labels: {lbl_caminho}")
    
    X_df = pd.read_csv(attr_caminho)
    y_df = pd.read_csv(lbl_caminho)

    # converte para vetor
    X = X_df.values
    y = y_df.values.flatten() 

    num_classes = len(np.unique(y))
    num_features = X.shape[1]
    
    print(f"Dataset carregado com sucesso:")
    print(f" - Amostras: {X.shape[0]}")
    print(f" - Features (Timestamps): {num_features}")
    print(f" - Classes únicas encontradas: {num_classes} {np.unique(y)}")

    # One-hot para redes neurais
    y_categorical = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    
    return X, y_categorical, y, num_features, num_classes


### constroi_mlp

Define as arquiteturas das MLP usadas no artigo. A função recebe o nome da configuração, a forma de entrada (timestamps do dataset) e número de classes de saída. 

**models.Sequential()**: cria um objeto do tipo modelo sequencial -> camadas da rede serão empilhadas uma após a outra em ordem. 

**model.add(layers.InputLayer(shape=shape))**: define camada de entrada com o formato de entrada dos dados. O argumento shape informa a rede o número de features que ela receberá.

**estrutura do if**: o if definirá a camada oculta com base no nome da configuração. Se for X neurônios, model.add(layers.Dense(X, activation='relu')) adiciona uma camada densa com X neurônios e a função de ativação ReLU. A parte de 16-8N é uma topologia multi-camada, ou seja, há duas camadas ocultas: uma densa com 16 neurônios e outra densa com 8 neurônios, ambas com função de ativação ReLU. Por fim, a camada de saída é comum a todas as redes com o número de neurônios exatamente igual ao número de classes (8). A função de ativação dessa camada é a Softmax, que transforma os scores internos em probabilidades para cada classe.

Por fim, a função *constroi_mlp()* retorna o objeto model (ou seja, retorna um modelo de aprendizado) completo e pronto para treino.

In [12]:
# MODELOS (Tópico 3.4)

def constroi_mlp(config_name, shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(shape=shape))
    
    if config_name == '4N':
        model.add(layers.Dense(4, activation='relu'))
    elif config_name == '8N':
        model.add(layers.Dense(8, activation='relu'))
    elif config_name == '16N':
        model.add(layers.Dense(16, activation='relu'))
    elif config_name == '32N':
        model.add(layers.Dense(32, activation='relu'))
    elif config_name == '16-8N':
        model.add(layers.Dense(16, activation='relu'))
        model.add(layers.Dense(8, activation='relu'))
        
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model


### constroi_cnn

A função define as arquiteturas das CNN usadas no artigo. Recebe o nome da configuração (Mx), o formato da entrada (no nosso caso, 201 timestamps de pressão do dataset) e o número de classes (8 no nosso caso).

**models.Sequential()**: cria uma pilha linear de camadas em que os dados fluem sequencialmente de uma camada para outra.

**model.add(layers.InputLayer(shape=shape))**: a rede receberá um vetor com a forma informada na função, ou seja, receberá um vetor de 201 pontos de pressão regulada conforme nosso dataset.

**bloco if**: primeiramente seleciona a configuração base Mx. A base da M1 até a M4 é a seguinte, em que **A**, **B** e **C** são parâmetros fornecidos pelo artigo:
```python
if config == Mx, em que x = número de 1 a 4:
    model.add(layers.Conv1D(filters=**A**, kernel_size=**B**, padding='same')) 
    # Adiciona camada de convolução 1D. Aplica **A** filtro(s) deslizante(s) com o tamanho de janela de **B** pontos temporais. Isso extrai características locais da curva de presão. 
    model.add(layers.AveragePooling1D(pool_size=**C**))
    # Adiciona a camada de pooling reduzindo a dimensão dos dados tirando média a cada **C** pontos.
    model.add(layers.Flatten())
    # Adiciona camada de achatamento transformando a matriz resultante das convoluções em um vetor longo e único para entrar na camada densa
    model.add(layers.Dense(16, activation='relu'))
    # Por fim, adiciona camada densa intermediária antes da fully-connected com processamento de 16 neurônios e ReLU de ativação, sendo um padrão em todas as CNNs testadas no artigo.
    # A camada M5 possui um segundo bloco de camada de convolução 1D e pooling, ou seja, a rede é mais profunda. 
model.add(layers.Dense(num_classes, activation='softmax')) # Todas as camadas Mx, da 1 até a 5, recebem a última camada de saída com o número de neurônios = número de classes com ativação Softmax.
```

**Diferenças das configurações M1 até M5**:
- M1: configuração base das CNNs testadas, com 1 filtro deslizante, tamanho de janela de 8 pontos temporais e redução de dimensão de dados a cada 4 pontos.
- M2: os autores testaram se aumentar o número de filtros deslizantes de 1 para 2 melhora a detecção de características, com o resto igual a M1.
- M3: o artigo testa se uma janela de observação maior de 16 pontos temporais captura melhores padrões, com o resto igual a M1.
- M4: aumenta o número da redução da dimensão de dados de 4 para 8, com o resto igual a M1.
- M5: adiciona segundo bloco de convolução 1D e pooling, testando se uma rede mais profunda com duas camadas de convolução melhora o desempenho.

Por fim, a função retorna o modelo de aprendizado construído.

In [ ]:
def constroi_cnn(config_name, shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(shape=shape))
    
    if config_name == 'M1':
        model.add(layers.Conv1D(filters=1, kernel_size=8, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=4))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))
        
    elif config_name == 'M2':
        model.add(layers.Conv1D(filters=2, kernel_size=8, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=4))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))
        
    elif config_name == 'M3':
        model.add(layers.Conv1D(filters=1, kernel_size=16, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=4))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))
        
    elif config_name == 'M4':
        model.add(layers.Conv1D(filters=1, kernel_size=8, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=8))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))
        
    elif config_name == 'M5':
        model.add(layers.Conv1D(filters=1, kernel_size=8, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=4))
        model.add(layers.Conv1D(filters=1, kernel_size=8, padding='same'))
        model.add(layers.AveragePooling1D(pool_size=2))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))

    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

### Experimentação

Primeiramente há o carregamento dos dados por meio da função carrega_dados() anteriormente discutida. Há o retorno das variáveis:
- **X**: matriz com as 8000 amostras de pressão.
- **y_cat**: rótulos das categorias de classificação no formato one-hot encoded.
- **y_integers**: rótulos das categorias em formato original para usar na estratificação da validação cruzada.
- **n_features**: número de momentos diferentes da mesma pressão, no caso, 201.
- **n_classes**: quantas classes o problema de classificação possui.

**Separação do dataset**: é por meio da função train_test_split(). A função recebe o dataset inteiro (**X**), os rótulos das categorias hot encoded (**y_cat**), os rótulos das categorias inteiros (**y_integers**), o tamanho do dataset de teste pretendido, nesse caso, 10% ou seja, **test_size = 0.10**. O dataset de teste nunca encontrará o dataset de treino, essa função garante que a informação testada nunca tenha sido reconhecida pelo modelo anteriormente. O atributo **stratify** da função garante a amostragem estratificada mantendo a mesma proporção de falhas tanto no treino quanto no teste. Ele recebe y_integers como valor, ou seja, identifica-se quantas classes devem ser garantidas na mesma porcentagem. O **random_state = 0.42** garante que a divisão seja sempre a mesma. Por fim, a função devolve o dataset de treino em **X_train**, o dataset de teste em **X_test**, as classes de treino em **y_train**, as classes de teste em **y_test** tanto em hot encoded quanto em inteiros (**y_train_int** e **y_test_int**).

**Preparação de dados para a CNN**: as matrizes de dados **X_train** e **X_test** não estão no formato esperado pelas camadas convolucionais (Conv1D) da biblioteca Keras, que exigem uma entrada tridimensional (amostras, timesteps, canais). Por isso, utiliza-se a função reshape() para adicionar uma dimensão extra de tamanho 1 ao final, representando o único canal da série temporal. As novas variáveis **X_train_cnn** e **X_test_cnn** assumem o formato (amostras, 201, 1).

**Definição dos formatos de entrada**: são criadas as tuplas **mlp_shape** e **cnn_shape** para informar à primeira camada de cada rede qual o formato dos dados que entrarão. Para o MLP é um vetor plano de (201,) e para a CNN é a estrutura de (201, 1).

**Definição de Hiperparâmetros**: são estabelecidas as constantes que guiarão o processo de treinamento:
- **EPOCHS** = 50: define que o modelo passará pelo dataset completo 50 vezes durante o treino.
- **BATCH_SIZE** = 32: indica que os pesos da rede serão atualizados a cada 32 amostras processadas.
- **K_FOLDS** = 10: define que a validação cruzada dividirá os dados de treino em 10 partes distintas.

**Configuração da Validação Cruzada**: inicializa-se uma lista vazia resultados para armazenar as métricas de cada rodada. O objeto **skf** é instanciado usando a classe StratifiedKFold. Ele é configurado com **n_splits=10** (para dividir em 10 partes), **shuffle=True** (para embaralhar os dados antes da divisão e evitar viés de ordem) e **random_state=42** (para garantir a reprodutibilidade dos folds). Esse objeto não divide os dados imediatamente, mas define a lógica que será usada no loop de treinamento.

In [24]:
# EXPERIMENTOS
    
X, y_cat, y_integers, n_features, n_classes = carrega_dados(ATTRIBUTES_FILE, LABELS_FILE)

# divisao treino/teste (10% Teste)
X_train, X_test, y_train, y_test, y_train_int, y_test_int = train_test_split(
    X, y_cat, y_integers, test_size=0.10, stratify=y_integers, random_state=42
)

# CNN: (samples, timesteps, features=1)
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

mlp_shape = (n_features,)
cnn_shape = (n_features, 1)

EPOCHS = 50 
BATCH_SIZE = 32
K_FOLDS = 10

resultados = []
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

print(f"\nIniciando Validação Cruzada (K={K_FOLDS})...")

Lendo atributos: ./dataset/attributes.csv
Lendo labels: ./dataset/label.csv
Dataset carregado com sucesso:
 - Amostras: 8000
 - Features (Timestamps): 201
 - Classes únicas encontradas: 8 [0 1 2 3 4 5 6 7]

Iniciando Validação Cruzada (K=10)...


### Rodando as arquiteturas MLP

**Avaliação das topologias MLP**: define-se a lista **configuracoes_mlp** contendo as chaves das arquiteturas propostas no artigo, variando de '4N' até '16-8N'. O código itera sobre cada configuração **cfg** para realizar a validação cruzada completa.

**Loop de Validação Cruzada**: utiliza-se o método **skf.split()** aplicado aos dados de treino (**X_train**) e seus rótulos inteiros (**y_train_int**). Esse método gera, para cada uma das 10 iterações (folds), os índices específicos de treino (**train_idx**) e validação (**val_idx**). Com base nesses índices, os dados são fatiados criando os subconjuntos de treino do fold (**X_fold_train**, **y_fold_train**) e os de validação do fold (**X_fold_val**, **y_fold_val**).

**Construção e Treinamento**: para cada fold, uma nova instância limpa do modelo é instanciada via **constroi_mlp()**, recebendo a configuração atual e o formato de entrada **mlp_shape**. O modelo é compilado definindo o otimizador como **adam**, a função de perda como **categorical_crossentropy** e a métrica de avaliação como **accuracy**. O treinamento é executado via **model.fit()** utilizando os dados do fold atual e as constantes **EPOCHS** e **BATCH_SIZE** previamente definidas, com **verbose=0** para suprimir a saída de logs no terminal.

**Coleta de Métricas e Agregação**: imediatamente após o treino, o modelo é submetido aos dados de validação através de **model.evaluate()**, e a acurácia obtida é armazenada na lista temporária **rel_acuracia**. Ao término dos 10 folds para uma configuração, calculam-se a média (**mean_acc**) e o desvio padrão (**std_acc**) das acurácias observadas. Esses valores consolidados são adicionados à lista **resultados**, registrando o tipo de modelo ('MLP'), a configuração testada e suas métricas de validação.

In [ ]:
# MLP
configuracoes_mlp = ['4N', '8N', '16N', '32N', '16-8N']
for cfg in configuracoes_mlp:
    print(f"Avaliando MLP: {cfg}")
    rel_acuraacia = []
    for train_idx, val_idx in skf.split(X_train, y_train_int):
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

        model = constroi_mlp(cfg, mlp_shape, n_classes)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_fold_train, y_fold_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        _, acc = model.evaluate(X_fold_val, y_fold_val, verbose=0)
        rel_acuraacia.append(acc)
        
    mean_acc = np.mean(rel_acuraacia)
    std_acc = np.std(rel_acuraacia)
    resultados.append({'Model': 'MLP', 'Config': cfg, 'Val_Acc_Mean': mean_acc, 'Val_Acc_Std': std_acc})
    print(f"  -> Acurácia Média: {mean_acc:.4f}")
    print(f"  -> Desvio Médio: {std_acc:.4f}")


Avaliando MLP: 4N
  -> Acurácia Média: 0.9921
  -> Desvio Médio: 0.0040
Avaliando MLP: 8N
  -> Acurácia Média: 0.9950
  -> Desvio Médio: 0.0029
Avaliando MLP: 16N


### Rodando as arquiteturas CNN

**Avaliação das topologias CNN**: estabelece-se a lista **configuracoes_cnn** com as variações de arquitetura descritas no estudo, indo de 'M1' a 'M5'. O loop principal percorre cada configuração **cfg** para submetê-la ao processo rigoroso de validação cruzada.

**Loop de Validação Cruzada**: repete-se a lógica de divisão utilizada anteriormente com o método **skf.split()**, garantindo que os mesmos índices de folds sejam usados, o que permite uma comparação justa entre MLP e CNN. A diferença crucial ocorre no fatiamento dos dados: as variáveis **X_fold_train** e **X_fold_val** são extraídas da matriz **X_train_cnn** (previamente redimensionada para 3 dimensões), enquanto os rótulos **y_fold_train** e **y_fold_val** continuam vindo do vetor original **y_train**.

**Construção e Treinamento**: a cada iteração do fold, o modelo é instanciado pela função **constroi_cnn()**, que recebe a configuração atual e o formato de entrada específico **cnn_shape** (201, 1). A compilação mantém a consistência metodológica utilizando o otimizador **adam** e a função de perda **categorical_crossentropy**. O ajuste dos pesos ocorre via **model.fit()** respeitando os hiperparâmetros globais **EPOCHS** e **BATCH_SIZE**, novamente com a supressão de logs (**verbose=0**).

**Coleta de Métricas e Agregação**: o desempenho do modelo no fold é medido através de **model.evaluate()**, salvando a acurácia na lista **rel_acuraacia**. Ao final do ciclo de 10 folds, computam-se a média (**mean_acc**) e o desvio padrão (**std_acc**) para aquela configuração. Os dados são estruturados e anexados à lista geral **resultados**, desta vez identificando o modelo como 'CNN', permitindo a comparação direta de performance e estabilidade com os resultados obtidos pelos modelos MLP.

In [ ]:
# CNN
confiiguracoes_cnn = ['M1', 'M2', 'M3', 'M4', 'M5']
for cfg in confiiguracoes_cnn:
    print(f"Avaliando CNN: {cfg}")
    rel_acuraacia = []
    for train_idx, val_idx in skf.split(X_train, y_train_int):
        X_fold_train, X_fold_val = X_train_cnn[train_idx], X_train_cnn[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

        model = constroi_cnn(cfg, cnn_shape, n_classes)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_fold_train, y_fold_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        _, acc = model.evaluate(X_fold_val, y_fold_val, verbose=0)
        rel_acuraacia.append(acc)
        
    mean_acc = np.mean(rel_acuraacia)
    std_acc = np.std(rel_acuraacia)
    resultados.append({'Model': 'CNN', 'Config': cfg, 'Val_Acc_Mean': mean_acc, 'Val_Acc_Std': std_acc})
    print(f"  -> Acurácia Média: {mean_acc:.4f}")
    print(f"  -> Desvio Médio: {std_acc:.4f}")


Avaliando CNN: M1
  -> Acc Média: 0.9944
Avaliando CNN: M2
  -> Acc Média: 0.9933
Avaliando CNN: M3
  -> Acc Média: 0.9957
Avaliando CNN: M4
  -> Acc Média: 0.9943
Avaliando CNN: M5
  -> Acc Média: 0.9925


### Rodando a KNN

**Avaliação do KNN**: define-se a lista **k_values** contendo os hiperparâmetros de vizinhos a serem testados, variando de 1 a 20, conforme estipulado no artigo para comparação com as redes neurais. O loop itera sobre cada valor **k** para realizar a validação cruzada.

**Loop de Validação Cruzada**: utiliza-se novamente o objeto **skf.split()** para garantir que os folds sejam idênticos aos experimentos anteriores. Uma distinção importante ocorre aqui: enquanto as redes neurais utilizavam rótulos one-hot encoded, o KNN utiliza os vetores de rótulos inteiros. Portanto, as variáveis de alvo **y_fold_train_labels** e **y_fold_val_labels** são extraídas diretamente de **y_train_int** (formato original 0-7) com base nos índices do fold atual.

**Construção e Treinamento**: para cada fold, instancia-se o classificador **KNeighborsClassifier** configurado com o número de vizinhos **n_neighbors=k** da iteração atual. O método **knn.fit()** é chamado para ajustar o modelo aos dados de treino do fold. Diferente das redes neurais que requerem múltiplas épocas, o KNN é um algoritmo de aprendizado preguiçoso (*lazy learning*), onde o "treinamento" consiste basicamente no armazenamento dos dados para cálculos de distância posteriores.

**Coleta de Métricas e Agregação**: a avaliação é feita através de **knn.score()**, que retorna diretamente a acurácia do modelo nos dados de validação. Esse valor é acumulado na lista **rel_acuraacia**. Ao final dos 10 folds, calculam-se a média (**mean_acc**) e o desvio padrão (**std_acc**). O dicionário de resultados é atualizado com o identificador 'KNN', o valor de K testado e as métricas estatísticas consolidadas.

In [ ]:

# KNN
k_values = [1, 2, 5, 10, 20]
print("\nAvaliando KNN...")
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    rel_acuraacia = []
    for train_idx, val_idx in skf.split(X_train, y_train_int):
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train_labels = y_train_int[train_idx]
        y_fold_val_labels = y_train_int[val_idx]
        knn.fit(X_fold_train, y_fold_train_labels)
        acc = knn.score(X_fold_val, y_fold_val_labels)
        rel_acuraacia.append(acc)
    
    mean_acc = np.mean(rel_acuraacia)
    std_acc = np.std(rel_acuraacia)
    resultados.append({'Model': 'KNN', 'Config': f'K={k}', 'Val_Acc_Mean': mean_acc, 'Val_Acc_Std': std_acc})
    print(f"  -> KNN K={k}: {mean_acc:.4f}")


Avaliando KNN...
  -> KNN K=1: 0.8826
  -> KNN K=2: 0.8553
  -> KNN K=5: 0.8862
  -> KNN K=10: 0.8753
  -> KNN K=20: 0.8597


### Comparando arquiteturas 

**Seleção do Melhor Modelo**: os dados coletados durante os loops de validação são convertidos em um DataFrame do pandas (**resultados_df**) para facilitar a manipulação. A melhor configuração global é identificada automaticamente através do método **.idxmax()** aplicado à coluna **'Val_Acc_Mean'**. Isso seleciona a linha que obteve a maior acurácia média entre todos os experimentos (MLP, CNN e KNN), armazenando as informações do vencedor na variável **melhor_rodada**.

**Retreinamento e Avaliação Final**: com a arquitetura vencedora definida, o código entra em uma estrutura condicional para instanciar o **modelo_final**. É crucial notar que, nesta etapa, o modelo é retreinado utilizando **todo o conjunto de treino** (**X_train** e **y_train**), aproveitando 100% dos dados disponíveis para aprendizado, e não apenas parciais dos folds.
* Para **MLP** ou **CNN**: o modelo é reconstruído do zero com a função construtora correspondente e a configuração vencedora. Após a compilação, o treinamento final ocorre (respeitando **EPOCHS** e **BATCH_SIZE**). A avaliação definitiva da capacidade de generalização é realizada aplicando o método **.evaluate()** sobre o conjunto de teste reservado (**X_test** ou **X_test_cnn**), computando a **final_acc**.
* Para **KNN**: o valor de *k* é extraído da string de configuração. O classificador é ajustado aos dados de treino inteiros e a acurácia é calculada diretamente via **.score()** usando os dados de teste.

[cite_start]**Relatório Final**: o script exibe no console a **Acurácia Final no Teste**, que é a métrica mais importante do estudo[cite: 137, 138], representando o desempenho real do modelo em dados novos e nunca vistos (os 10% separados no início). Por fim, imprime-se o DataFrame completo, permitindo uma análise comparativa da média e desvio padrão entre todas as topologias testadas.

In [ ]:

# Resultado Final
resultados_df = pd.DataFrame(resultados)
melhor_rodada = resultados_df.loc[resultados_df['Val_Acc_Mean'].idxmax()]

print("\n" + "="*40)
print(f"MELHOR MODELO: {melhor_rodada['Model']} ({melhor_rodada['Config']})")
print("="*40)

final_acc = 0
if melhor_rodada['Model'] == 'MLP':
    modelo_final = constroi_mlp(melhor_rodada['Config'], mlp_shape, n_classes)
    modelo_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    modelo_final.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    loss, final_acc = modelo_final.evaluate(X_test, y_test, verbose=0)
elif melhor_rodada['Model'] == 'CNN':
    modelo_final = constroi_cnn(melhor_rodada['Config'], cnn_shape, n_classes)
    modelo_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    modelo_final.fit(X_train_cnn, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    loss, final_acc = modelo_final.evaluate(X_test_cnn, y_test, verbose=0)
elif melhor_rodada['Model'] == 'KNN':
    k = int(melhor_rodada['Config'].split('=')[1])
    modelo_final = KNeighborsClassifier(n_neighbors=k)
    modelo_final.fit(X_train, y_train_int)
    final_acc = modelo_final.score(X_test, y_test_int)

print(f"Acurácia Final no Teste (10%): {final_acc:.4f}")
print("\nResumo Completo:")
print(resultados_df[['Model', 'Config', 'Val_Acc_Mean', 'Val_Acc_Std']])



MELHOR MODELO: MLP (16N)
Acurácia Final no Teste (10%): 0.9950

Resumo Completo:
   Model Config  Val_Acc_Mean  Val_Acc_Std
0    MLP     4N      0.989167     0.005336
1    MLP     8N      0.995556     0.002307
2    MLP    16N      0.996667     0.002927
3    MLP    32N      0.995694     0.002668
4    MLP  16-8N      0.994167     0.004598
5    CNN     M1      0.994444     0.003106
6    CNN     M2      0.993333     0.003768
7    CNN     M3      0.995694     0.002519
8    CNN     M4      0.994306     0.001577
9    CNN     M5      0.992500     0.003298
10   KNN    K=1      0.882639     0.017070
11   KNN    K=2      0.855278     0.011782
12   KNN    K=5      0.886250     0.013909
13   KNN   K=10      0.875278     0.007212
14   KNN   K=20      0.859722     0.010282
15   KNN    K=1      0.882639     0.017070
16   KNN    K=2      0.855278     0.011782
17   KNN    K=5      0.886250     0.013909
18   KNN   K=10      0.875278     0.007212
19   KNN   K=20      0.859722     0.010282
